In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

In [2]:
# Vuoksi Macro Lab: Labor–Leisure Tradeoff Supreme™ (Charles Jones+ Edition)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.edgecolor'] = 'gray'

# --- Fully Enhanced Labor-Leisure Tradeoff Model ---
def labor_leisure_goated(w=10, T=24, theta=1.0):
    ℓ_star = theta / (1 + theta) * T
    L_star = T - ℓ_star
    c_star = w * L_star

    ℓ_vals = np.linspace(0.1, T-0.1, 500)
    c_vals = w * (T - ℓ_vals)

    def utility(c, ℓ):
        return np.log(c) + theta * np.log(ℓ)

    U_star = utility(c_star, ℓ_star)
    U_levels = [U_star - 0.5, U_star, U_star + 0.5]

    fig, ax = plt.subplots(figsize=(10, 7))

    # Budget Line
    ax.plot(ℓ_vals, c_vals, label='Budget Line: $c = w(T - ℓ)$', color='black', linewidth=2)

    # Indifference Curves
    for i, U in enumerate(U_levels):
        C = np.exp(U - theta * np.log(ℓ_vals))
        label = f"U = {U:.2f}" if i == 1 else None
        ax.plot(ℓ_vals, C, linestyle='--', color='gray', alpha=0.6, label=label)

    # Optimal Point
    ax.scatter(ℓ_star, c_star, color='red', s=100, label='Optimal Bundle')
    ax.annotate(f"(ℓ* = {ℓ_star:.1f}, c* = {c_star:.1f})", xy=(ℓ_star, c_star),
                xytext=(ℓ_star + 1, c_star + 5), arrowprops=dict(arrowstyle='->'), fontsize=12)

    # Tangency Condition Text
    ax.text(0.05 * T, 0.95 * max(c_vals), "At optimum: MRS = w",
            fontsize=12, bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', ec='gray'))

    ax.set_xlabel("Leisure (ℓ)")
    ax.set_ylabel("Consumption (c)")
    ax.set_title("Labor–Leisure Tradeoff with Indifference Curves and Tangency")
    ax.set_xlim(0, T)
    ax.set_ylim(0, max(c_vals) * 1.1)
    ax.legend()
    ax.grid(True)
    plt.tight_layout()
    plt.show()

    print("\033[1mRESULTS:\033[0m")
    print(f"Optimal Leisure ℓ*: {ℓ_star:.2f} hrs")
    print(f"Optimal Labor L*: {L_star:.2f} hrs")
    print(f"Consumption c*: ${c_star:.2f}")
    print(f"Utility U(c, ℓ): {U_star:.3f}")
    print("Condition: Marginal Rate of Substitution (MRS) = Wage (w)")

interact(
    labor_leisure_goated,
    w=FloatSlider(value=10, min=1, max=50, step=1, description='Wage (w)'),
    T=FloatSlider(value=24, min=8, max=24, step=1, description='Time Endowment (T)'),
    theta=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Pref. Leisure (θ)'),
);


interactive(children=(FloatSlider(value=10.0, description='Wage (w)', max=50.0, min=1.0, step=1.0), FloatSlide…